In [1]:
!pip install nltk bs4 tqdm

In [2]:
import re
import string
import pandas as pd
from bs4 import BeautifulSoup
from tqdm.auto import tqdm
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag, word_tokenize
from nltk.corpus import wordnet
tqdm.pandas()
stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

In [3]:

nltk.download("punkt")
nltk.download("wordnet")
nltk.download("stopwords")
nltk.download("omw-1.4")

# POS tagger models (new + legacy)
nltk.download("averaged_perceptron_tagger")
nltk.download("averaged_perceptron_tagger_eng")

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


True

In [4]:
#JD data
data = pd.read_csv("/kaggle/input/job-description-dataset/job_descriptions.csv")
jdf = pd.DataFrame(data)
jdf

,Job Id,Experience,Qualifications,Salary Range,location,Country,latitude,longitude,Work Type,Company Size,...,Contact,Job Title,Role,Job Portal,Job Description,Benefits,skills,Responsibilities,Company,Company Profile
0,1089843540111562,5 to 15 Years,M.Tech,$59K-$99K,Douglas,Isle of Man,54.2361,-4.5481,Intern,26801,...,001-381-930-7517x737,Digital Marketing Specialist,Social Media Manager,Snagajob,Social Media Managers oversee an organizations...,"{'Flexible Spending Accounts (FSAs), Relocatio...","Social media platforms (e.g., Facebook, Twitte...","Manage and grow social media accounts, create ...",Icahn Enterprises,"{""Sector"":""Diversified"",""Industry"":""Diversifie..."
1,398454096642776,2 to 12 Years,BCA,$56K-$116K,Ashgabat,Turkmenistan,38.9697,59.5563,Intern,100340,...,461-509-4216,Web Developer,Frontend Web Developer,Idealist,Frontend Web Developers design and implement u...,"{'Health Insurance, Retirement Plans, Paid Tim...","HTML, CSS, JavaScript Frontend frameworks (e.g...","Design and code user interfaces for websites, ...",PNC Financial Services Group,"{""Sector"":""Financial Services"",""Industry"":""Com..."
2,481640072963533,0 to 12 Years,PhD,$61K-$104K,Macao,"Macao SAR, China",22.1987,113.5439,Temporary,84525,...,9687619505,Operations Manager,Quality Control Manager,Jobs2Careers,Quality Control Managers establish and enforce...,"{'Legal Assistance, Bonuses and Incentive Prog...",Quality control processes and methodologies St...,Establish and enforce quality control standard...,United Services Automobile Assn.,"{""Sector"":""Insurance"",""Industry"":""Insurance: P..."
3,688192671473044,4 to 11 Years,PhD,$65K-$91K,Porto-Novo,Benin,9.3077,2.3158,Full-Time,129896,...,+1-820-643-5431x47576,Network Engineer,Wireless Network Engineer,FlexJobs,"Wireless Network Engineers design, implement, ...","{'Transportation Benefits, Professional Develo...",Wireless network design and architecture Wi-Fi...,"Design, configure, and optimize wireless netwo...",Hess,"{""Sector"":""Energy"",""Industry"":""Mining, Crude-O..."
4,117057806156508,1 to 12 Years,MBA,$64K-$87K,Santiago,Chile,-35.6751,-71.5429,Intern,53944,...,343.975.4702x9340,Event Manager,Conference Manager,Jobs2Careers,A Conference Manager coordinates and manages c...,"{'Flexible Spending Accounts (FSAs), Relocatio...",Event planning Conference logistics Budget man...,Specialize in conference and convention planni...,Cairn Energy,"{""Sector"":""Energy"",""Industry"":""Energy - Oil & ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1615935,134563577088850,0 to 12 Years,B.Tech,$64K-$114K,"Malabo (de jure),",Equatorial Guinea,1.6508,10.2679,Full-Time,18281,...,950-451-5843,Mechanical Engineer,Mechanical Design Engineer,ZipRecruiter,Mechanical Design Engineers create and develop...,"{'Employee Assistance Programs (EAP), Tuition ...","Mechanical engineering CAD software (e.g., Sol...","Design mechanical systems, components, and pro...",The Hershey Company,"{""Sector"":""Food and Beverage/Confectionery"",""I..."
1615936,618604818190827,2 to 14 Years,M.Tech,$62K-$130K,Warsaw,Poland,51.9194,19.1451,Intern,63621,...,676.387.1572x71877,IT Manager,IT Director,USAJOBS,An IT Director oversees an organizations IT de...,"{'Health Insurance, Retirement Plans, Paid Tim...",Strategic IT planning Leadership and managemen...,Provide strategic leadership for IT department...,EQT,"{""Sector"":""Energy"",""Industry"":""Energy"",""City"":..."
1615937,615471367712200,4 to 15 Years,BCA,$60K-$96K,Ashgabat,Turkmenistan,38.9697,59.5563,Part-Time,114287,...,537.384.6193x5284,Mechanical Engineer,Mechanical Design Engineer,Indeed,Mechanical Design Engineers create and develop...,"{'Tuition Reimbursement, Stock Options or Equi...","Mechanical engineering CAD software (e.g., Sol...","Design mechanical systems, components, and pro...",KLA,"{""Sector"":""Technology"",""Industry"":""Semiconduct..."
1615938,804137342023945,5 to 15 Years,BCA,$65K-$103K,Ouagadougou,Burk

In [5]:
jdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1615940 entries, 0 to 1615939
Data columns (total 23 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   Job Id            1615940 non-null  int64  
 1   Experience        1615940 non-null  object 
 2   Qualifications    1615940 non-null  object 
 3   Salary Range      1615940 non-null  object 
 4   location          1615940 non-null  object 
 5   Country           1615940 non-null  object 
 6   latitude          1615940 non-null  float64
 7   longitude         1615940 non-null  float64
 8   Work Type         1615940 non-null  object 
 9   Company Size      1615940 non-null  int64  
 10  Job Posting Date  1615940 non-null  object 
 11  Preference        1615940 non-null  object 
 12  Contact Person    1615940 non-null  object 
 13  Contact           1615940 non-null  object 
 14  Job Title         1615940 non-null  object 
 15  Role              1615940 non-null  object 
 16  

In [6]:
jdf['Job Title'].unique()

array(['Digital Marketing Specialist', 'Web Developer',
       'Operations Manager', 'Network Engineer', 'Event Manager',
       'Software Tester', 'Teacher', 'UX/UI Designer', 'Wedding Planner',
       'QA Analyst', 'Litigation Attorney', 'Mechanical Engineer',
       'Network Administrator', 'Account Manager', 'Brand Manager',
       'Social Worker', 'Social Media Coordinator',
       'Email Marketing Specialist', 'HR Generalist', 'Legal Assistant',
       'Nurse Practitioner', 'Account Director', 'Software Engineer',
       'Purchasing Agent', 'Sales Consultant', 'Civil Engineer',
       'Network Security Specialist', 'UI Developer', 'Financial Planner',
       'Event Planner', 'Psychologist', 'Electrical Designer',
       'Data Analyst', 'Technical Writer', 'Tax Consultant',
       'Account Executive', 'Systems Administrator',
       'Database Administrator', 'Research Analyst', 'Data Entry Clerk',
       'Registered Nurse', 'Investment Analyst', 'Speech Therapist',
       'Sales M

In [7]:
#Filter the IT Job Title only 
it_keywords = [
    "developer", "engineer", "software", "data", "network", "system", 
    "administrator", "architect", "qa", "it", "scientist", "analyst", "support", "ui", "ux"
]

def is_it_job(title):
    title_lower = str(title).lower()
    return any(keyword in title_lower for keyword in it_keywords)
jdf = jdf[jdf["Job Title"].apply(is_it_job)].reset_index(drop=True)

In [8]:
jdf["Job Title"].unique()

array(['Digital Marketing Specialist', 'Web Developer',
       'Network Engineer', 'Software Tester', 'UX/UI Designer',
       'QA Analyst', 'Litigation Attorney', 'Mechanical Engineer',
       'Network Administrator', 'Nurse Practitioner', 'Software Engineer',
       'Civil Engineer', 'Network Security Specialist', 'UI Developer',
       'Data Analyst', 'Technical Writer', 'Systems Administrator',
       'Database Administrator', 'Research Analyst', 'Data Entry Clerk',
       'Investment Analyst', 'Landscape Architect', 'UX Researcher',
       'IT Support Specialist', 'Software Developer',
       'Supply Chain Analyst', 'Data Engineer', 'Electrical Engineer',
       'Systems Engineer', 'Inventory Analyst', 'Database Developer',
       'Systems Analyst', 'Copywriter', 'Content Writer',
       'Java Developer', 'Family Nurse Practitioner',
       'Front-End Engineer', 'Architectural Designer',
       'Financial Analyst', 'Environmental Engineer',
       'Back-End Developer', 'Structural

In [9]:
non =  ['Digital Marketing Specialist', 'Litigation Attorney','Nurse Practitioner','Civil Engineer','Technical Writer','Research Analyst','Investment Analyst', 'Landscape Architect', 'Supply Chain Analyst','Electrical Engineer','Inventory Analyst','Copywriter', 'Content Writer','Family Nurse Practitioner','Architectural Designer','Environmental Engineer', 'Financial Analyst', 'Structural Engineer','Market Research Analyst', 'Customer Support Specialist','Chemical Analyst', 'Process Engineer', 'Architect','Research Scientist', 'Aerospace Engineer', 'Marketing Analyst', 'SEO Analyst', 'Chemical Engineer', 'Market Analyst']
jdf = jdf[~jdf['Job Title'].isin(non)].reset_index(drop=True)

In [10]:
jdf['Job Title'].unique()

array(['Web Developer', 'Network Engineer', 'Software Tester',
       'UX/UI Designer', 'QA Analyst', 'Mechanical Engineer',
       'Network Administrator', 'Software Engineer',
       'Network Security Specialist', 'UI Developer', 'Data Analyst',
       'Systems Administrator', 'Database Administrator',
       'Data Entry Clerk', 'UX Researcher', 'IT Support Specialist',
       'Software Developer', 'Data Engineer', 'Systems Engineer',
       'Database Developer', 'Systems Analyst', 'Java Developer',
       'Front-End Engineer', 'Back-End Developer', 'Business Analyst',
       'Network Technician', 'Software Architect', 'IT Manager',
       'Quality Assurance Analyst', 'Front-End Developer', 'QA Engineer',
       'Data Scientist', 'Network Analyst', 'IT Administrator'],
      dtype=object)

In [11]:
cols = ["Job Id","Job Title","Role","Job Description","Responsibilities","skills","Experience","Qualifications"]

# Filter the dataframe
jdf = jdf[cols]

In [12]:
jdf

,Job Id,Job Title,Role,Job Description,Responsibilities,skills,Experience,Qualifications
0,398454096642776,Web Developer,Frontend Web Developer,Frontend Web Developers design and implement u...,"Design and code user interfaces for websites, ...","HTML, CSS, JavaScript Frontend frameworks (e.g...",2 to 12 Years,BCA
1,688192671473044,Network Engineer,Wireless Network Engineer,"Wireless Network Engineers design, implement, ...","Design, configure, and optimize wireless netwo...",Wireless network design and architecture Wi-Fi...,4 to 11 Years,PhD
2,116831420231957,Software Tester,Quality Assurance Analyst,A Quality Assurance Analyst tests software and...,Test software applications and systems to iden...,Quality assurance processes Testing methodolog...,4 to 12 Years,MCA
3,1498778686197107,UX/UI Designer,User Interface Designer,User Interface Designers focus on the visual a...,Create visually appealing user interfaces (UI)...,UI design principles and best practices Graphi...,2 to 8 Years,M.Com
4,1680293940995740,UX/UI Designer,Interaction Designer,Interaction Designers specialize in designing ...,"Work on interaction design, defining how users...",Interaction design principles User behavior an...,2 to 9 Years,BBA
...,...,...,...,...,...,...,...,...
404086,1827059374762454,UX/UI Designer,User Interface Designer,User Interface Designers focus on the visual a...,Create visually appealing user interfaces (UI)...,UI design principles and best practices Graphi...,5 to 12 Years,B.Com
404087,1162938205421273,UI Developer,UX/UI Designer,A UX/UI Designer focuses on enhancing user exp...,"Design user interfaces and experiences, focusi...",User experience (UX) design User interface (UI...,4 to 14 Years,B.Com
404088,134563577088850,Mechanical Engineer,Mechanical Design Engineer,Mechanical Design Engineers create and develop...,"Design mechanical systems, components, and pro...","Mechanical engineering CAD software (e.g., Sol...",0 to 12 Years,B.Tech
404089,618604818190827,IT Manager,IT Director,An IT Director oversees an organizations IT de...,Provide strategic leadership for IT department...,Strategic IT planning Leadership and managemen...,2 to 14 Years,M.Tech


In [13]:
#Resume data
data = pd.read_csv("/kaggle/input/resume-dataset/Resume/Resume.csv")
rdf = pd.DataFrame(data)
rdf

,ID,Resume_str,Resume_html,Category
0,16852973,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,"<div class=""fontsize fontface vmargins hmargin...",HR
1,22323967,"HR SPECIALIST, US HR OPERATIONS ...","<div class=""fontsize fontface vmargins hmargin...",HR
2,33176873,HR DIRECTOR Summary Over 2...,"<div class=""fontsize fontface vmargins hmargin...",HR
3,27018550,HR SPECIALIST Summary Dedica...,"<div class=""fontsize fontface vmargins hmargin...",HR
4,17812897,HR MANAGER Skill Highlights ...,"<div class=""fontsize fontface vmargins hmargin...",HR
...,...,...,...,...
2479,99416532,RANK: SGT/E-5 NON- COMMISSIONED OFFIC...,"<div class=""fontsize fontface vmargins hmargin...",AVIATION
2480,24589765,"GOVERNMENT RELATIONS, COMMUNICATIONS ...","<div class=""fontsize fontface vmargins hmargin...",AVIATION
2481,31605080,GEEK SQUAD AGENT Professional...,"<div class=""fontsize fontface vmargins hmargin...",AVIATION
2482,21190805,PROGRAM DIRECTOR / OFFICE MANAGER ...,"<div class=""fontsize fontface vmargins hmargin...",AVIATION


In [14]:
rdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2484 entries, 0 to 2483
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ID           2484 non-null   int64 
 1   Resume_str   2484 non-null   object
 2   Resume_html  2484 non-null   object
 3   Category     2484 non-null   object
dtypes: int64(1), object(3)
memory usage: 77.8+ KB


In [15]:
rdf['Category'].unique()

array(['HR', 'DESIGNER', 'INFORMATION-TECHNOLOGY', 'TEACHER', 'ADVOCATE',
       'BUSINESS-DEVELOPMENT', 'HEALTHCARE', 'FITNESS', 'AGRICULTURE',
       'BPO', 'SALES', 'CONSULTANT', 'DIGITAL-MEDIA', 'AUTOMOBILE',
       'CHEF', 'FINANCE', 'APPAREL', 'ENGINEERING', 'ACCOUNTANT',
       'CONSTRUCTION', 'PUBLIC-RELATIONS', 'BANKING', 'ARTS', 'AVIATION'],
      dtype=object)

In [16]:
rdf = rdf[rdf['Category'] == 'INFORMATION-TECHNOLOGY']

In [17]:
rdf

,ID,Resume_str,Resume_html,Category
217,36856210,INFORMATION TECHNOLOGY Summar...,"<div class=""fontsize fontface vmargins hmargin...",INFORMATION-TECHNOLOGY
218,21780877,INFORMATION TECHNOLOGY SPECIALIST\tGS...,"<div class=""fontsize fontface vmargins hmargin...",INFORMATION-TECHNOLOGY
219,33241454,INFORMATION TECHNOLOGY SUPERVISOR ...,"<div class=""fontsize fontface vmargins hmargin...",INFORMATION-TECHNOLOGY
220,25990239,INFORMATION TECHNOLOGY INSTRUCTOR ...,"<div class=""fontsize fontface vmargins hmargin...",INFORMATION-TECHNOLOGY
221,16899268,INFORMATION TECHNOLOGY MANAGER/ANALYS...,"<div class=""fontsize fontface vmargins hmargin...",INFORMATION-TECHNOLOGY
...,...,...,...,...
332,27485716,CORPORATE PROJECT MANAGER Caree...,"<div class=""fontsize fontface vmargins hmargin...",INFORMATION-TECHNOLOGY
333,26480367,IT TECHNOLOGY SPECIALIST Profes...,"<div class=""fontsize fontface vmargins hmargin...",INFORMATION-TECHNOLOGY
334,81761658,IT MANAGER Highlights ...,"<div class=""fontsize fontface vmargins hmargin...",INFORMATION-TECHNOLOGY
335,79541391,SUBJECT MATTER EXPERT (INFORMATION TE...,"<div class=""fontsize fontface vmargins hmargin...",INFORMATION-TECHNOLOGY


In [18]:
rdf = rdf.drop(columns=["ID", "Resume_html"])

In [19]:
cols = list(rdf.columns)
if "Resume_str" in cols and "Category" in cols:
    cols.remove("Resume_str")
    cols.remove("Category")
    new_order = ["Category", "Resume_str"] + cols
    rdf = rdf[new_order]


In [20]:
rdf=rdf.rename(columns={"Resume_str":"Resume"})

In [21]:
rdf

,Category,Resume
217,INFORMATION-TECHNOLOGY,INFORMATION TECHNOLOGY Summar...
218,INFORMATION-TECHNOLOGY,INFORMATION TECHNOLOGY SPECIALIST\tGS...
219,INFORMATION-TECHNOLOGY,INFORMATION TECHNOLOGY SUPERVISOR ...
220,INFORMATION-TECHNOLOGY,INFORMATION TECHNOLOGY INSTRUCTOR ...
221,INFORMATION-TECHNOLOGY,INFORMATION TECHNOLOGY MANAGER/ANALYS...
...,...,...
332,INFORMATION-TECHNOLOGY,CORPORATE PROJECT MANAGER Caree...
333,INFORMATION-TECHNOLOGY,IT TECHNOLOGY SPECIALIST Profes...
334,INFORMATION-TECHNOLOGY,IT MANAGER Highlights ...
335,INFORMATION-TECHNOLOGY,SUBJECT MATTER EXPERT (INFORMATION TE...


In [22]:
data1 = pd.read_csv('/kaggle/input/d/gauravduttakiit/resume-dataset/UpdatedResumeDataSet.csv')
rdf1 = pd.DataFrame(data1)
rdf1

,Category,Resume
0,Data Science,Skills * Programming Languages: Python (pandas...
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...
2,Data Science,"Areas of Interest Deep Learning, Control Syste..."
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab..."
...,...,...
957,Testing,Computer Skills: â¢ Proficient in MS office (...
958,Testing,â Willingness to accept the challenges. â ...
959,Testing,"PERSONAL SKILLS â¢ Quick learner, â¢ Eagerne..."
960,Testing,COMPUTER SKILLS & SOFTWARE KNOWLEDGE MS-Power ...


In [23]:
rdf1["Category"].unique()

array(['Data Science', 'HR', 'Advocate', 'Arts', 'Web Designing',
       'Mechanical Engineer', 'Sales', 'Health and fitness',
       'Civil Engineer', 'Java Developer', 'Business Analyst',
       'SAP Developer', 'Automation Testing', 'Electrical Engineering',
       'Operations Manager', 'Python Developer', 'DevOps Engineer',
       'Network Security Engineer', 'PMO', 'Database', 'Hadoop',
       'ETL Developer', 'DotNet Developer', 'Blockchain', 'Testing'],
      dtype=object)

In [24]:
rdf1.duplicated().sum()

796

In [25]:
rdf1=rdf1.drop_duplicates()

In [26]:
non1 =  ['HR','Advocate', 'Arts','Mechanical Engineer', 'Sales', 'Health and fitness','Civil Engineer', 'Electrical Engineering','Operations Manager']
rdf1 = rdf1[~rdf1["Category"].isin(non1)].reset_index(drop=True)

In [27]:
rdf1

,Category,Resume
0,Data Science,Skills * Programming Languages: Python (pandas...
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...
2,Data Science,"Areas of Interest Deep Learning, Control Syste..."
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab..."
...,...,...
104,Testing,Computer Skills: â¢ Proficient in MS office (...
105,Testing,â Willingness to accept the challenges. â ...
106,Testing,"PERSONAL SKILLS â¢ Quick learner, â¢ Eagerne..."
107,Testing,COMPUTER SKILLS & SOFTWARE KNOWLEDGE MS-Power ...


In [28]:
rdf_concat = pd.concat([rdf, rdf1], ignore_index=True)

In [29]:
#Resume dataset
rdf_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 229 entries, 0 to 228
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  229 non-null    object
 1   Resume    229 non-null    object
dtypes: object(2)
memory usage: 3.7+ KB


In [30]:
URL_RE   = re.compile(r"https?://\S+|www\.\S+")
EMAIL_RE = re.compile(r"\S+@\S+")
PHONE_RE = re.compile(r"(\+?\d[\d\-\s\(\)]{6,}\d)")
MULTI_SPACE_RE = re.compile(r"\s+")
# allow +, #, . and - inside tokens if desired, but will simplify later
NON_ALPHANUMERIC_RE = re.compile(r"[^A-Za-z0-9\+\#\.\-\s]")

def strip_html(text):
    if pd.isna(text):
        return ""
    return BeautifulSoup(str(text), "lxml").get_text(separator=" ")

def basic_clean(text, remove_digits=False):
    if pd.isna(text):
        return ""
    s = str(text)
    s = strip_html(s)
    s = URL_RE.sub(" ", s)
    s = EMAIL_RE.sub(" ", s)
    s = PHONE_RE.sub(" ", s)
    s = NON_ALPHANUMERIC_RE.sub(" ", s)
    if remove_digits:
        s = re.sub(r"\d+", " ", s)
    s = MULTI_SPACE_RE.sub(" ", s)
    return s.strip().lower()

In [31]:
def penn_to_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    if tag.startswith('V'):
        return wordnet.VERB
    if tag.startswith('N'):
        return wordnet.NOUN
    if tag.startswith('R'):
        return wordnet.ADV
    return wordnet.NOUN

In [32]:
def tokenize_lemmatize(text, keep_pos=None, keep_only_tech=False, tech_vocab=None, min_len=2):
    if not text:
        return ""
    tokens = word_tokenize(text)  # NLTK tokenizer
    # POS tagging
    pos_tags = pos_tag(tokens)
    out_tokens = []
    for tok, tag in pos_tags:
        tok_lower = tok.lower().strip()
        if tok_lower in stop_words:
            continue
        if len(tok_lower) < min_len:
            continue
        pos = penn_to_wordnet_pos(tag)
        lemma = lemmatizer.lemmatize(tok_lower, pos=pos)
        lemma = lemma.strip()
        if not lemma:
            continue
        if keep_only_tech:
            if tech_vocab is None:
                continue
            # exact-match to tech vocab (vocab should be lowercased)
            if lemma in tech_vocab:
                out_tokens.append(lemma)
        else:
            out_tokens.append(lemma)
    return " ".join(out_tokens)


In [33]:
def build_tech_vocab(df, skills_col="skills"):
    vocab = set()
    if skills_col not in df.columns:
        return vocab
    for s in df[skills_col].dropna().astype(str):
        parts = re.split(r"[,\|;/•\n]+", s)
        for p in parts:
            token = p.strip().lower()
            token = re.sub(r"[^a-z0-9\+\#\.\- ]", "", token)
            token = token.replace(".", "")
            if token:
                vocab.add(token)
    return vocab

tech_vocab = build_tech_vocab(jdf, skills_col="skills")   
len(tech_vocab)

181

In [34]:
def process_in_chunks(df, col, out_col, func, chunk_size=50000, **func_kwargs):
    n = len(df)
    results = []
    out_series = pd.Series(index=df.index, dtype=object)
    for start in range(0, n, chunk_size):
        end = min(start + chunk_size, n)
        idx = df.index[start:end]
        chunk = df.loc[idx, col].fillna("").astype(str)
        # apply elementwise
        processed = chunk.progress_apply(lambda x: func(x, **func_kwargs))
        out_series.loc[idx] = processed
        print(f"Processed rows {start}..{end-1}")
    df.loc[:, out_col] = out_series
    return df


In [35]:
# ensure working copies
jdf = jdf.copy()
rdf_concat = rdf_concat.copy()  

# 7A. Basic clean (keeps digits so experience numbers remain — set remove_digits=True if you want to drop numbers)
for col in ["Job Title", "Job Description", "skills", "Responsibilities"]:
    if col in jdf.columns:
        jdf.loc[:, col + "_basic"] = jdf[col].progress_apply(lambda x: basic_clean(x, remove_digits=False))

# for resume column - detect the column name; adjust as needed
resume_col = "Resume" if "Resume" in rdf_concat.columns else ("Resume_str" if "Resume_str" in rdf_concat.columns else rdf_concat.columns[1])
rdf_concat.loc[:, resume_col + "_basic"] = rdf_concat[resume_col].progress_apply(lambda x: basic_clean(x, remove_digits=False))

# 7B. Lemmatize and filter in chunks for job description (keep general tokens)
jdf = process_in_chunks(jdf, col="Job Description_basic", out_col="Job_Description_proc",
                        func=tokenize_lemmatize, chunk_size=50000,
                        keep_pos=None, keep_only_tech=False, tech_vocab=tech_vocab)

# 7C. Skills -> keep only tech vocab (exact matches)
jdf = process_in_chunks(jdf, col="skills_basic", out_col="Skills_proc",
                        func=tokenize_lemmatize, chunk_size=100000,
                        keep_pos=None, keep_only_tech=True, tech_vocab=tech_vocab)

# 7D. Job Title (short) - process directly (smaller)
jdf.loc[:, "Job_Title_proc"] = jdf["Job Title_basic"].progress_apply(
    lambda x: tokenize_lemmatize(x, keep_only_tech=False, tech_vocab=tech_vocab)
)

# 7E. Responsibilities (optional)
if "Responsibilities_basic" in jdf.columns:
    jdf = process_in_chunks(jdf, col="Responsibilities_basic", out_col="Responsibilities_proc",
                            func=tokenize_lemmatize, chunk_size=50000,
                            keep_pos=None, keep_only_tech=False, tech_vocab=tech_vocab)

# 7F. Resumes
rdf_concat = process_in_chunks(rdf_concat, col=resume_col + "_basic", out_col=resume_col + "_proc",
                        func=tokenize_lemmatize, chunk_size=20000,
                        keep_pos=None, keep_only_tech=False, tech_vocab=tech_vocab)


  0%|          | 0/404091 [00:00<?, ?it/s]

  0%|          | 0/404091 [00:00<?, ?it/s]

  0%|          | 0/404091 [00:00<?, ?it/s]

  0%|          | 0/404091 [00:00<?, ?it/s]

  0%|          | 0/229 [00:00<?, ?it/s]

...

Processed rows 350000..399999


  0%|          | 0/4091 [00:00<?, ?it/s]

Processed rows 400000..404090


  0%|          | 0/229 [00:00<?, ?it/s]

Processed rows 0..228


In [36]:
def make_combined_job_text(row, title_proc="Job_Title_proc", skills_proc="Skills_proc", desc_proc="Job_Description_proc", max_chars=3000):
    parts = []
    t = row.get(title_proc) or row.get("Job Title") or ""
    if isinstance(t, str) and t.strip():
        parts.append(t.strip())
    s = row.get(skills_proc) or ""
    if s:
        parts.append(s)
    d = row.get(desc_proc) or ""
    if d:
        parts.append(d)
    text = " | ".join([p for p in parts if p])
    return text[:max_chars]

jdf.loc[:, "combined_text"] = jdf.apply(make_combined_job_text, axis=1)
# For resumes, use processed resume text
rdf_concat.loc[:, "combined_text"] = rdf_concat[resume_col + "_proc"]

In [37]:
def clean_resume(text):
    if not isinstance(text, str):
        return ""
    
    # normalize whitespace + remove \xa0
    text = re.sub(r"\s+", " ", text).replace("\xa0", " ").strip().lower()
    
    # remove section headers (optional, can keep if useful)
    headers = ["summary", "skills", "experience", "education"]
    for h in headers:
        text = re.sub(rf"\b{h}\b", " ", text)
    
    # tokenize
    tokens = nltk.word_tokenize(text)
    
    # remove stopwords + non-alphabetic
    tokens = [t for t in tokens if t.isalpha() and t not in stop_words]
    
    # lemmatize
    tokens = [lemmatizer.lemmatize(t) for t in tokens]
    
    return " ".join(tokens)

In [38]:
resume_col = "Resume"  # adjust if different column name

rdf_concat[resume_col + "_proc"] = rdf_concat[resume_col].apply(clean_resume)

In [39]:
def make_combined_resume_text(row, resume_proc="Resume_proc", skills_col=None, max_chars=3000):
    parts = []
    if skills_col and row.get(skills_col):
        parts.append(row[skills_col])
    r = row.get(resume_proc) or ""
    if r:
        parts.append(r)
    text = " | ".join([p for p in parts if p])
    return text[:max_chars]

rdf_concat.loc[:, "combined_text"] = rdf_concat.apply(make_combined_resume_text, axis=1)
